In [1]:
! pip install -U bitsandbytes
! pip install langchain-huggingface
! pip install --upgrade langchain pydantic
! pip install -U langchain-community
! pip install docarray selenium
! pip langchain-text-splitters 
! pip install unstructured
! pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import SeleniumURLLoader
from langchain.chains import ConversationalRetrievalChain
from kaggle_secrets import UserSecretsClient



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-04-21 13:01:13.519084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745240473.930156      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745240474.052620      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Authenticate with Hugging Face Hub to access models
login(token =  UserSecretsClient().get_secret("HF_Token")) 

In [4]:
# Quantization Configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [5]:
# Load tokenizer 
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")  
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad_token is set

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=quantization_config,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
# First create the transformers pipeline
hf_pipeline = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 512,
    do_sample = False,
    repetition_penalty = 1.03,
    return_full_text = False,
)

# Then create the LangChain wrapper
llm = HuggingFacePipeline(pipeline = hf_pipeline)



Device set to use cuda:0


In [7]:
# Load webpage using Selenium 
loader = SeleniumURLLoader(urls = ["https://attackontitan.fandom.com/wiki/Eren_Yeager"])
docs = loader.load()

In [8]:
# Split long documents into smaller chunks for processing
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

In [9]:
# Create embeddings (vector representations of text)
embeddings = HuggingFaceEmbeddings()

# Create in-memory vector store
db = DocArrayInMemorySearch.from_documents(
    all_splits, 
    embeddings
)

# Create retriever interface
retriever = db.as_retriever() 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
INFO - docarray - DB config created
INFO - docarray - Runtime config created
INFO - docarray - No docs or index file provided. Initializing empty InMemoryExactNNIndex.


In [10]:
# Create a simple RetrievalQA chain with LLM and retriever
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    chain_type = "stuff", # Just "stuff" all the retrieved documents into one prompt
    verbose = False
)

In [11]:
qa_stuff.invoke("When and where was Eren Yeager born ?")

{'query': 'When and where was Eren Yeager born ?',
 'result': ' Eren Yeager was born in Shiganshina District on March 30th, 835.'}

In [12]:
qa_stuff.invoke("Who was Eren's best friend ?")  

{'query': "Who was Eren's best friend ?",
 'result': ' Eren had a deep bond with Armin Arlert, who he considered his best friend. They grew up together and Eren went to great lengths to protect Armin, even attempting to save him from human traffickers who had captured him. Their friendship was so strong that Eren would entrust his life to Armin without hesitation. However, their friendship was strained due to differing opinions on how to deal with Marley and the other nations.'}

In [13]:
qa_stuff.invoke( "Who were Eren's siblings ?")  

{'query': "Who were Eren's siblings ?",
 'result': ' Eren had an older half-brother named Zeke Yeager.'}

In [14]:
qa_stuff.invoke("Why did Eren admire the survey corps so much ?")

{'query': 'Why did Eren admire the survey corps so much ?',
 'result': ' Eren admired the Survey Corps because they were the only people who ventured beyond the Walls to explore the outside world, which he longed to see for himself. He saw them as heroes and wished to join their ranks as soon as he became eligible.'}